In [7]:
import pandas as pd
import numpy as np
import ast
import redis
import json
from web3 import Web3
from web3.middleware import geth_poa_middleware

# connect Redis databases
WATCHDOG_ADDRESS ="10.2.1.9"
db1 = redis.StrictRedis(
    host=WATCHDOG_ADDRESS,
    port=6379,
    db=1)
db1_keys = db1.keys()

db2 = redis.StrictRedis(
    host=WATCHDOG_ADDRESS,
    port=6379,
    db=2)
db2_keys = db2.keys()
genesis = ast.literal_eval(db2.get(b'genesis').decode('utf-8'))

# construct dataframe
rows = []
for key in db1_keys:
    row = []
    # IP address of node
    ip = key.decode("utf-8")
    # check if node is validator
    value = ast.literal_eval(db1.get(key).decode('utf-8'))
    is_validator = value['key'][2:] in genesis['extraData']
    # find the number of peers
    url = 'http://'+ip+':8545'
    provider = Web3.HTTPProvider(url)
    w3 = Web3(provider)
    w3.middleware_onion.inject(geth_poa_middleware, layer=0)
    peers_num = len(w3.geth.admin.peers())

    row.append(value['hostname'])
    row.append(ip)
    row.append(value['key'])
    row.append(is_validator)
    row.append(peers_num)
    
    rows.append(row)
df_orig = pd.DataFrame(np.array(rows),
    columns=['Hostname', 'IP', 'NodeAddress', 'IsValidator', 'PeersNum'])

# sort dataframe
df = df_orig.copy()
idx = []
for name in df.Hostname:
    idx.append(int(name.split('-')[1]))
df['Index'] = idx
df = df.set_index(keys=df.Index).drop(labels='Index', axis=1).sort_index()
df

,Hostname,IP,NodeAddress,IsValidator,PeersNum
Index,,,,,
1,besu-1,10.2.12.142,0x025499b012a23247f94fe0d73be9055bd8e5160d,True,5
2,besu-2,10.2.9.46,0xbce336f846afe819f5e1b7717b9c0ab5ba3cf3bf,True,3
3,besu-3,10.2.12.169,0xca4d7234dc963160c31567777c6e7cd7dc4b62a5,True,4
4,besu-4,10.2.5.143,0xb91fd5e6b28786ebe47b272c023d5944ccbaa9d8,True,4
